In [1]:
# Set your token here from https://dunya.compmusic.upf.edu/user/profile/
token = '2bd80bef6e90945815fe53660eb43faebe2346ac'

In [2]:
import codecs
import json, os, sys
import pickle
import csv
import time
import datetime
import collections

import numpy as np

import compmusic

from compmusic import dunya
from compmusic import dunya as dn
from compmusic.dunya import hindustani as hi
from compmusic.dunya import carnatic as ca
from compmusic.dunya import docserver as ds
from compmusic import musicbrainz

dn.set_token(token)

In [3]:
#Features list
features_dunya_all = [{'type': 'pitch', 'subtype': 'pitch', 'extension': '.pitch', 'version': 'noguessunv'},
                         {'type': 'ctonic', 'subtype': 'tonic', 'extension': '.tonic', 'version': '0.3'},
                         {'type': 'sama-manual', 'subtype': None, 'extension': '.sama', 'version': None},
                         {'type': 'sections-manual', 'subtype': None, 'extension': '.sections', 'version': None},
                         {'type': 'tempo-manual', 'subtype': None, 'extension': '.tempo', 'version': None},
                         {'type': 'pitch-vocal', 'subtype': None, 'extension': '.mpitch', 'version': None},
                         {'type': 'mphrases-manual', 'subtype': None, 'extension': '.mphrases', 'version': None},
                         {'type': 'sections-manual-p', 'subtype': None, 'extension': '.sections_p', 'version': None},
                         {'type': 'bpm-manual', 'subtype': None, 'extension': '.bpm', 'version': None}
                         ]

In [4]:
def get_stats_hindustani(dunya_collections = None):
    """Get information about hindustani recordings and return a summary of attributes.
    For the following attributes:
        release
        works
        raags
        taals
        forms
        layas
        album artists
        artists (musicians)
    generate a list of identifiers for these attributes (mbid or uuid [raag, taal, laya] or name [form])
    present in the collection
    
    Args:
        dunya_collections: a list of MusicBrainz/Dunya Collection IDs to restrict the Dunya API to
    """
    
    hi.set_collections(dunya_collections)
    recordings = hi.get_recordings()

    stats = collections.defaultdict(list)
    for r in recordings:
        mbid = r['mbid']

        try:
            rec_info = hi.get_recording(mbid)

            stats['release'].append([r['mbid'] for r in rec_info.get('release', [])])
            stats['works'].append([w['mbid'] for w in rec_info.get('works', [])])
            stats['raags'].append([r['uuid'] for r in rec_info.get('raags', [])])
            stats['taals'].append([t['uuid'] for t in rec_info.get('taals', [])])
            stats['forms'].append([f['name'] for f in rec_info.get('forms', [])])
            stats['layas'].append([l['uuid'] for l in rec_info.get('layas', [])])
            stats['album_artists'].append([a['mbid'] for a in rec_info.get('album_artists', [])])
            stats['artists'].append([a['artist']['mbid'] for a in rec_info.get('artists', [])])
            stats['length'].append(rec_info.get('length'))
        except:
            failure += 1
            print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
            print("Failed to fetch info for recording %s" % mbid) 
            print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    
    # Filter empty lists from the stats
    for k, vals in stats.items():
        stats[k] = [v for v in vals if v]
    
    return stats

def get_stats_carnatic(dunya_collections = None):
    """Get information about carnatic recordings and return a summary of attributes.
    For the following attributes:
        concert
        work
        raaga
        taala
        form
        album artists
        artists (musicians)
    generate a list of identifiers for these attributes (mbid or uuid [raaga, taala] or name [form])
    present in the collection
    
    Args:
        dunya_collections: a list of MusicBrainz/Dunya Collection IDs to restrict the Dunya API to
    """
    ca.set_collections(dunya_collections)
    recordings = ca.get_recordings()
    
    stats = collections.defaultdict(list)
    for r in recordings:
        mbid = r['mbid']

        try:
            rec_info = ca.get_recording(mbid)

            stats['concert'].append([c['mbid'] for c in rec_info.get('concert', [])])
            stats['work'].append([w['mbid'] for w in rec_info.get('work', [])])
            stats['raaga'].append([r['uuid'] for r in rec_info.get('raaga', [])])
            stats['taala'].append([t['uuid'] for t in rec_info.get('taala', [])])
            stats['form'].append([f['name'] for f in rec_info.get('form', [])])
            stats['album_artists'].append([a['mbid'] for a in rec_info.get('album_artists', [])])
            stats['artists'].append([a['artist']['mbid'] for a in rec_info.get('artists', [])])
            stats['length'].append(rec_info.get('length'))
        except dn.HTTPError:
            print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
            print("Failed to fetch info for recording %s" % mbid) 
            print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
        
    # Filter empty lists from the stats
    for k, vals in stats.items():
        stats[k] = [v for v in vals if v]
    
    return stats

    
def save_stats(stats, stats_file, summary_file):
    """Write statistics to file
    Args:
        stats (dict): the statistics to write
        stats_file (str): file path to write the statistics summary
        summary_file (str): file path to write a readable statistics summary
    """
    
    output_stats = {}
    for k, v in stats.items():
        if k == 'length':
            output_stats[k] = {'total_length': np.sum(v), 'total_recs': len(v)}
        else:
            output_stats[k] = {'total_unique': len(np.unique(sum(v, []))), 'unique_elems': np.unique(sum(v, [])).tolist(), 'total_rels': len(sum(v, [])), 'total_recs': len(v)}
    pickle.dump(output_stats, codecs.open(stats_file, 'wb'))
    
    
    with codecs.open(summary_file, 'w') as fp:
        for key1, val in output_stats.items():
            fp.write('------------ %s ------------\n'%str(key1))
            if key1 == 'length':
                for key2, val2 in val.items():
                    fp.write('%s\t%f\n'%(str(key2), float(val2)/(1000.0*3600.0)))
            else:
                for key2, val2 in val.items():
                    if key2 == 'unique_elems':
                        fp.write('%s\t%d\n'%(str(key2), len(val2)))
                    else:
                        fp.write('%s\t%d\n'%(str(key2), val2))
            fp.write('\n')

def save_sections(content, output_file):
    """
    This function saves the content(section annotations) into a file in a structured manner
    Annotations are already stored nicely but due to differences in the delimiters of Hindustani and Carnatic
    we needed this function
    
    Args:
        content (str): data read from dunya api
        output_file (str): file path for output file
    Outputs:
        Saves statistics to a text file
    """
    
    # detecting delimiter automatically
    snf = csv.Sniffer()
    delimiter = snf.sniff(content).delimiter
    rows = [k.split(delimiter) for k in content.split('\n') if k != '']
    csv.writer(output_file, rows, delimiter = '\t')


def download_files_for_collection(collection_name, collection_ids, features, numFiles = 5):
    """Download all files of a collection
    Args:
        collection (dict): dictionary containig name and id of the collection
        features (list of dicts): feature types
        numFiles (int): the maximum number of files to download
    Returns:
        A dictionary counting how many files for each feature was unable to be downloaded
    Outputs:
        Saves mp3 and annotation files of the collection
    """
    dataDir = collection_name
    os.makedirs(dataDir, exist_ok=True)

    if collection_name == 'hindustani':
        tradition = hi
    elif collection_name == 'carnatic':
        tradition = ca
    
    tradition.set_collections(collection_ids)
    recs = tradition.get_recordings()
    
    numFiles = min(numFiles, len(recs))
    
    print('Number of files in collection {}: {}'.format(collection_name, len(recs)))
    print('...will download {} files'.format(numFiles))
    
    # Creating data structure for keeping list of missing files
    missingData = collections.Counter()
    for feature in features:
        missingData[feature['type']] = 0
    
    # Downloading data
    for i, recording in enumerate(recs[:numFiles], 1):
        mbid = recording['mbid']
        print('{}/{}: {}'.format(i, len(recs), mbid))
        mp3_filename = tradition.download_mp3(mbid, dataDir)
        json_file = mp3_filename.replace('.mp3', '.json')
        with open(os.path.join(dataDir, json_file), 'w') as outfile:
            json.dump(tradition.get_recording(mbid), outfile)
        
        print(mp3_filename)

        for feature in features:
            try:
                content = ds.file_for_document(mbid, feature['type'], feature['subtype'], version=feature['version'])
                
                out_file = os.path.join(dataDir, mp3_filename.replace('.mp3','.{}.txt'.format(feature['type'])))
                if feature['type'] == 'pitch':
                    content = json.loads(content.decode())
                    content = np.array(content)
                    np.savetxt(out_file, content, fmt='%.7f', delimiter='\t')
                #elif feature['type'] == 'sections-manual' or feature['type'] == 'sections-manual-p':
                #    save_sections(content.decode(), out_file)
                else:
                    with open(out_file, 'w') as fp:
                        fp.write(content.decode())
            except dn.HTTPError:
                #print('Does not have ',feature['type'],' content for :',mbid)
                missingData[feature['type']] += 1
    
    print('Collection download finished.')
    print('----------------------------------------------------------')
    return dict(missingData)

In [5]:
DUNYA_COLLECTIONS = {'hindustani': ['6adc54c6-6605-4e57-8230-b85f1de5be2b'],
                     'carnatic': ['a163c8f2-b75f-4655-86be-1504ea2944c2']
                    }

In [7]:
# Calling functions to produce statistics and download data

missingDatas = {}
NUM_FILES = 1 #set to 200 if you like to download all data (CAUTION: 8Gb)

print('Starting process: {}'.format(datetime.datetime.now()))
print('Collecting statistics of carnatic collection')
carnatic_stats = get_stats_carnatic(DUNYA_COLLECTIONS['carnatic'])
output_file = 'stats_carnatic_cc.pkl'
output_file_pretty = 'stats_carnatic_cc.txt'
save_stats(carnatic_stats, output_file, output_file_pretty)

#print('Downloading files ... ')
#missingData = download_files_for_collection('carnatic', DUNYA_COLLECTIONS['carnatic'], features_dunya_all, NUM_FILES)
#missingDatas['carnatic'] = missingData

#print('...Done')
#print('Collecting statistics of hindustani collection')

#hindustani_stats = get_stats_hindustani(DUNYA_COLLECTIONS['hindustani'])
#output_file = 'stats_hindustani_cc.pkl'
#output_file_pretty = 'stats_hindustani_cc.txt'
#save_stats(hindustani_stats, output_file, output_file_pretty)

#print('Downloading files ... ')
#missingData = download_files_for_collection('hindustani', DUNYA_COLLECTIONS['hindustani'], features_dunya_all, NUM_FILES)
#missingDatas['hindustani'] = missingData
#print('...Done')

pickle.dump(missingDatas, codecs.open('missingData1.pkl', 'wb'))
print('Missing data list stored in missingData.pkl')

print('Finished! {}'.format(datetime.datetime.now()))

Starting process: 2020-05-29 10:22:45.024528
Missing data list stored in missingData.pkl
Finished! 2020-05-29 10:24:31.917647


In [3]:
instrument_ids_list=[10] #Flute, mandolin, saxophone
h=0
for iid in instrument_ids_list:
    for artist in dunya.carnatic.get_instrument(iid)['artists']:
        dataDir=dunya.carnatic.get_instrument(iid)['name']
        os.makedirs(dataDir,exist_ok=True)
        os.makedirs(os.path.join(dataDir,'_json'),exist_ok=True)
        os.makedirs(os.path.join(dataDir,'_txt'),exist_ok=True)
        concerts = dunya.carnatic.get_artist(artist['mbid'])['concerts']
        print(concerts)

[{'mbid': 'dc08430d-4092-4d22-aeff-b488135169db', 'title': 'December Season 2006'}, {'mbid': 'c5d9d3bd-bc01-4104-b874-d55219bd0e54', 'title': 'Madrasil Margazhi 2006'}, {'mbid': '60f5bad7-992e-4999-812f-c3a19a2666c1', 'title': 'Oothukkadu Kritis'}]
[{'mbid': '1a757fe4-852a-4e17-b656-9eaac20d9f8f', 'title': 'December Season 2000'}]
[{'mbid': '729225dc-2255-4642-b744-65da7f4887b1', 'title': 'Paddhatti - M M Dhandapani Desigar Live Concert - 1964 - Vol I'}]
[{'mbid': '9a76e90c-cf9c-4691-b93f-02109ad93d6d', 'title': 'December Season 2004'}, {'mbid': 'c9a86282-e5c2-4ef0-8caf-1f0f1e4e58a6', 'title': 'December Season 2008'}, {'mbid': '1b295774-e382-4b16-9e0b-ec0508edfeab', 'title': 'Pravrddha Sri Lalgudi Pancharatna Kritis'}, {'mbid': 'c9a96cc4-b512-47dd-88ad-ddad21c12d71', 'title': 'December Season 2009'}, {'mbid': '394da7e5-910e-4e4b-b99c-6452e497d903', 'title': 'December Season 2010'}, {'mbid': 'bac2a810-50e9-4bd1-abfd-1c7850481525', 'title': 'Gems Of Carnatic Music - Live In Concert 2006'

In [17]:
instrument_ids_list=[3] #Flute, mandolin, saxophone
h=0
for iid in instrument_ids_list:
    for artist in dunya.carnatic.get_instrument(iid)['artists']:
        dataDir=dunya.carnatic.get_instrument(iid)['name']
        os.makedirs(dataDir,exist_ok=True)
        os.makedirs(os.path.join(dataDir,'_json'),exist_ok=True)
        os.makedirs(os.path.join(dataDir,'_txt'),exist_ok=True)
        concerts = dunya.carnatic.get_artist(artist['mbid'])['concerts']
        print(concerts,'\n')
        #for i in compmusic.dunya.carnatic.get_concert(concerts['mbid'])['recordings'][25:26]:
            #print(i)

[{'mbid': '50ccb84d-9bab-4497-ad3e-404ef10c8595', 'title': 'Flute'}, {'mbid': 'e469c5ae-0e48-4ff8-8276-ef81d352acf5', 'title': 'December Season 2000'}, {'mbid': 'c4fe6220-f5da-4cfa-aabe-6315fa7f538d', 'title': "Maestro's Choice: Vocal"}, {'mbid': '1eec130e-b7ba-4a82-8758-caf62cb42471', 'title': 'December Season 2006'}, {'mbid': '955c8fc1-3c89-4e22-a894-9ad0cccdbaa8', 'title': 'Dr S Sundar at Arkay'}] 

[{'mbid': '54ab1640-8479-4b7e-bb35-b40c30176501', 'title': 'Carnatic Vocal (1910 - 1965)'}] 

[{'mbid': 'fdec2654-e731-49bb-a84d-33c9258e162f', 'title': 'Vocal'}] 

[{'mbid': 'b7fb0f21-7fcf-407d-ad95-89531de42e4f', 'title': 'December Season 2001'}, {'mbid': '8cceac2e-53d2-4ab4-b614-78b654955ab0', 'title': 'Madrasil Margazhi 2006'}, {'mbid': '6be52cc9-4ba9-45e2-9a72-42791234c064', 'title': 'December Season 2007'}, {'mbid': 'c5d9d3bd-bc01-4104-b874-d55219bd0e54', 'title': 'Madrasil Margazhi 2006'}, {'mbid': 'c70cd697-cfd1-4a80-a448-0a14d17364bd', 'title': "Memorable Concerts From Rfa's Sep

[{'mbid': 'b93ad376-058a-4278-9aab-8427d88b6145', 'title': 'Live At Sree Rama Seva Mandali, Bangalore'}] 

[{'mbid': 'fc48231b-b8db-42a6-afed-1c336dd1541a', 'title': 'Paddhatti'}, {'mbid': '3c31557f-bfbb-4561-b40d-499016ecbadd', 'title': 'Paddhatti'}, {'mbid': 'c969a7c1-ab5c-4eb5-bd60-825c32318325', 'title': 'Master Piece - Live Concert at Sri Krishna Gana Sabha Vol.III'}, {'mbid': '9a76e90c-cf9c-4691-b93f-02109ad93d6d', 'title': 'December Season 2004'}, {'mbid': '2fba8ade-1f09-4198-98e4-a9396aa4f6e4', 'title': 'Collectors Choice'}] 

[{'mbid': 'c9a86282-e5c2-4ef0-8caf-1f0f1e4e58a6', 'title': 'December Season 2008'}, {'mbid': 'c9a96cc4-b512-47dd-88ad-ddad21c12d71', 'title': 'December Season 2009'}] 

[{'mbid': 'f0170257-349b-4fe2-980b-4dc320eb81d6', 'title': 'Paddhatti'}, {'mbid': '08d7b7e1-e9c8-40fd-b505-7c381e9848eb', 'title': 'AIR Recitals 1987'}] 

[{'mbid': 'ae6acbe9-1320-4e1d-a687-52080717eaca', 'title': 'December Season 1999'}] 

[{'mbid': 'ff1579a7-9063-4ea7-ae95-758a31c701ed',

[{'mbid': '60e9a3be-89fd-4b88-8408-dfebd8bce461', 'title': 'Tanjore S KALYANARAMAN Live in concert-1972-vol I and vol II'}] 

[{'mbid': 'cc02d86c-2528-432a-8091-930526df3a50', 'title': 'Paddhatti'}] 

[{'mbid': '41b2e362-9066-4a75-870a-61813665bad7', 'title': 'Paddhatti'}, {'mbid': 'f6b1e607-e15f-41d9-ae68-3246cebd770b', 'title': 'Paddhatti'}] 

[{'mbid': 'd6cec350-d9ed-4649-9be6-f6a6d4cae9bd', 'title': 'Paddhatti'}] 

[] 

[] 

[{'mbid': 'c5dd06f4-b0f5-4f40-b335-2db7928b3ae4', 'title': 'KP Nandini at Arkay'}] 

[{'mbid': 'c43ac27c-e639-41ba-864b-4c0d3e3cf758', 'title': 'Mahati at Arkay'}] 

[{'mbid': 'bcabf61e-a9b5-445e-9f39-a9c71e09cc6e', 'title': 'Srividya Janakiraman at Arkay'}] 

[{'mbid': '1d48de77-e4bf-468e-bb42-97a8f518c077', 'title': 'Kuldeep Pai at Arkay'}] 

[{'mbid': '00bca904-80c5-47fb-9194-d3543d187be3', 'title': 'V Shankarnarayanan at Arkay'}] 

[{'mbid': 'b4bf9f9f-55a0-467e-8023-f6bd6df7606f', 'title': 'Paddhatti'}, {'mbid': 'aeac5f25-8799-48d8-abb2-b6f6c5257072', 'titl

In [20]:
dunya.carnatic.get_concert('0816586d-c83e-4c79-a0aa-9b0e578f408d')

{'mbid': '0816586d-c83e-4c79-a0aa-9b0e578f408d',
 'title': 'Cherthala Ranganatha Sharma at Arkay',
 'year': 2014,
 'image': 'https://dunya.compmusic.upf.edu/media/missing/concert.jpg',
 'recordings': [{'mbid': 'a05b885f-59af-4766-a82e-2fea4578cf08',
   'title': 'Varashiki Vahana',
   'disc': 1,
   'disctrack': 1,
   'track': 1},
  {'mbid': '9f5a5452-14cb-4af0-9289-4833854ee60d',
   'title': 'Bhuvini Dasudane',
   'disc': 1,
   'disctrack': 2,
   'track': 2},
  {'mbid': '26ff1e93-2c1b-47dc-b40a-7f64504232a0',
   'title': 'Velum Mayilume',
   'disc': 1,
   'disctrack': 3,
   'track': 3},
  {'mbid': '99ef06f0-38cf-456c-9541-a8a04edaae5f',
   'title': 'Karuna Nidhi Illalo',
   'disc': 1,
   'disctrack': 4,
   'track': 4},
  {'mbid': 'cc4199f2-ecf3-4d93-b628-65f2caacc6f8',
   'title': 'Tulasi Bilva',
   'disc': 1,
   'disctrack': 5,
   'track': 5},
  {'mbid': '2badb85a-4f7c-4933-9910-ff01a3968d08',
   'title': 'Rama Rama Guna Seema',
   'disc': 1,
   'disctrack': 6,
   'track': 6},
  {'mbid

duniya.carnatic.get_instruent(

In [10]:
for i in dunya.carnatic.get_instrument(10)['artists']:
    dunya.carnatic.get_concert(i['mbid'])

HTTPError: 404 Client Error: Not Found for url: https://dunya.compmusic.upf.edu/api/carnatic/concert/9359040e-7ca5-4cb6-85ab-bb90cd91a618?extra_headers=None

In [29]:
dunya.hindustani.get_instrument(10)['name']

HTTPError: 404 Client Error: Not Found for url: https://dunya.compmusic.upf.edu/api/hindustani/instrument/10

In [32]:
dunya.hindustani.get_instruments()

[{'mbid': 'd92884b7-ee0c-46d5-96f3-918196ba8c5b', 'name': 'Voice'},
 {'mbid': '117dacfc-0ad0-4e90-81a4-a28b4c03929b',
  'name': 'Spanish acoustic guitar'},
 {'mbid': '1ebfe130-b68c-452a-8ee3-81b430d13ca3', 'name': 'Bansuri'},
 {'mbid': '0fe1a768-45ba-49e4-8363-14db8e73ca85', 'name': 'Cymbals'},
 {'mbid': '540280f1-d6cf-46bf-968b-695e99e216d7', 'name': 'Flute'},
 {'mbid': 'c5aa7d98-c14d-4ff1-8afb-f8743c62496c', 'name': 'Ghatam'},
 {'mbid': '63021302-86cd-4aee-80df-2270d54f4978', 'name': 'Guitar'},
 {'mbid': 'c43c7647-077d-4d60-a01b-769de71b82f2', 'name': 'Harmonium'},
 {'mbid': 'f0a6e89d-b828-4b73-885d-8c1560e5e49a', 'name': 'Harp'},
 {'mbid': 'e2f0cdf4-3651-4bb9-9153-0cdb16ce65ea', 'name': 'Morsing'},
 {'mbid': 'f689271c-37bc-4c49-92a3-a14b15ee5d0e', 'name': 'Mridangam'},
 {'mbid': 'cf1343bf-0893-4a62-9c72-e25ee5be0309', 'name': 'Pakhavaj'},
 {'mbid': '9ace3b20-ff09-4a53-b737-79f9dc1ca90b', 'name': 'Santoor'},
 {'mbid': '79f5331d-d17c-4c2e-8ad0-44144432a754', 'name': 'Sarangi'},
 {'mbi

In [9]:
dunya.carnatic.get_instrument(10)['name']

'Khanjira'

In [13]:
for i in dunya.carnatic.get_instrument(10)['artists']:
    print(i)

{'mbid': '9359040e-7ca5-4cb6-85ab-bb90cd91a618', 'name': 'Trivandrum D. Rajagopal'}
{'mbid': '3627443c-5e83-4302-bbf3-07c0fe220475', 'name': 'Mayavaram G Somu'}
{'mbid': '9c268932-1f5e-4f81-9e8d-bd14f0de369c', 'name': 'C S Veerusami Pillai'}
{'mbid': 'd5f832d5-42ae-48da-a4e4-dd02cdb8059b', 'name': 'Abhishek Raghuram'}
{'mbid': 'ff942686-f728-45c2-bf29-140f09cdef57', 'name': 'R. Harishankar'}
{'mbid': '8ea4d1f9-a265-479b-b168-ade14dadf603', 'name': 'B S Purushottam'}
{'mbid': 'bf7c6c00-1568-4676-bdf7-0b6aa0f4fe1d', 'name': 'G Harishankar'}
{'mbid': '92e61dd5-9367-4878-b89c-5551bb802575', 'name': 'S Venkataraman'}
{'mbid': 'd8b187b3-333f-44b4-a42b-02312e9b3eba', 'name': 'Bangalore N. Amrit'}
{'mbid': '64c98753-d570-47ec-b5f6-0c0ac0f23811', 'name': 'Shankar'}
{'mbid': '6167a046-02ee-4fb2-a67e-350ae1becc46', 'name': 'V Nagarajan'}
{'mbid': '63b0d57f-efc4-4479-ab4c-0f8d2f9cedc4', 'name': 'K V Gopalakrishnan'}
{'mbid': '206cdcb7-0b09-4336-a49f-fafc62a472d7', 'name': 'Hari shankar'}
{'mbid': 

In [8]:
dunta.carnatic.get_artists('T.N Krishna')

NameError: name 'dunta' is not defined